<div style="text-align: justify; padding:5px; background-color:rgb(252, 253, 255); border: 1px solid lightgrey; padding-left: 1em; padding-right: 1em;">
    <font color='red'>To begin: Click anywhere in this cell and press <kbd>Run</kbd> on the menu bar. This executes the current cell and then highlights the next cell. There are two types of cells. A <i>text cell</i> and a <i>code cell</i>. When you <kbd>Run</kbd> a text cell (<i>we are in a text cell now</i>), you advance to the next cell without executing any code. When you <kbd>Run</kbd> a code cell (<i>identified by <span style="font-family: courier; color:black; background-color:white;">In[ ]:</span> to the left of the cell</i>) you advance to the next cell after executing all the Python code within that cell. Any visual results produced by the code (text/figures) are reported directly below that cell. Press <kbd>Run</kbd> again. Repeat this process until the end of the notebook. <b>NOTE:</b> All the cells in this notebook can be automatically excecuted sequentially by clicking <kbd>Kernel</kbd><font color='black'>→</font><kbd>Restart and Run All</kbd>. Should anything crash then restart the Jupyter Kernal by clicking <kbd>Kernel</kbd><font color='black'>→</font><kbd>Restart</kbd>, and start again from the top.
        
</div>

<div style="background-color:rgb(255, 250, 250); padding:5px; padding-left: 1em; padding-right: 1em;">
<img src="images/logo_text.png" width="200px" align="right"></p>

<h1 id="tuturial1basicmetabolomicsdataanalysisworkflow" style="text-align: justify">Tutorial 1: Basic Metabolomics Data Analysis Workflow</h1>

<p><br>
<br>
<br>
<br>

<p  style="text-align: justify">This Jupyter notebook describes a typical metabolomics data analysis workflow for a study with a binary classification outcome. The main steps included are: </p>

<ul>
<li style="text-align: justify">Import metabolite &amp; experimental data from an Excel sheet. </li>

<li style="text-align: justify">Pooled QC-based data cleaning.</li>

<li style="text-align: justify">Principal Component Analysis visualisation to check data quality.</li>

<li style="text-align: justify">Two-class univariate statistics.</li>

<li style="text-align: justify">Multivariate analysis using Partial Least Squares Discriminant Analysis (PLS-DA) including:


<ul>
<li style="text-align: justify">model optimisation (R<sup>2</sup> vs Q<sup>2</sup>).</li>

<li style="text-align: justify">permutation testing, model prediction metrics.</li>

<li style="text-align: justify">feature importance.</li>

<li style="text-align: justify">model prediction data visualisations.</li></ul>
</li>

<li style="text-align: justify">Export statistical tables to Excel sheets.</li>
</ul>

<p style="text-align: justify">The study used in this tutorial has been previously published as an open access article <a href="https://www.nature.com/articles/bjc2015414">Chan et al. (2016)</a>, in the British Journal of Cancer, and the deconvolved and annotated data file deposited at the <a href="http://www.metabolomicsworkbench.org">Metabolomics Workbench data repository</a> (Project ID PR000699). The data can be accessed directly via its project <a href="http://dx.doi.org/DOI:10.21228/M8B10B">DOI:10.21228/M8B10B</a> <sup>1</sup>H-NMR spectra were acquired at Canada’s National High Field Nuclear Magnetic Resonance Centre (NANUC) using a 600 MHz Varian Inova spectrometer. Spectral deconvolution and metabolite annotation was performed using the <a href="https://www.chenomx.com/software/">Chenomx NMR Suite v7.6</a>. Unfortunately, the Raw NMR data is unavailable.</p>

</div>

<div style="background-color:rgb(255, 250, 250); padding:5px; padding-left: 1em; padding-right: 1em;">
    
<h2 id="1importpackagesmodules" style="text-align: justify">1. Import Packages/Modules</h2>

<p style="text-align: justify">The first code cell of this tutorial (below this text box) imports <a href="https://docs.python.org/3/tutorial/modules.html"><em>packages</em> and <em>modules</em></a> into the Jupyter environment. <em>Packages</em> and <em>modules</em> provide additional functions and tools that extend the basic functionality of the Python language. We will need the following tools  to analyse the data in this tutorial:<br></p>

<ul>
<li style="text-align: justify"><a href="http://www.numpy.org/"><code>numpy</code></a>: the fundamental package for scientific computing with Python, providing tools to work with arrays and linear algebra</li>

<li style="text-align: justify"><a href="https://pandas.pydata.org/"><code>pandas</code></a>: provides high-performance, easy-to-use data structures and data analysis tools</li>

<li style="text-align: justify"><a href="http://beakerx.com/"><code>beakerx</code></a>: provides interactive tools for the Jupyter notebook environment</li>

<li style="text-align: justify"><a href="https://scikit-learn.org/stable/"><code>sklearn</code></a>: tools for machine learning in Python


<ul>
<li style="text-align: justify"><a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html"><code>train_test_split</code></a>: a method to split arrays into random test/training subsets for cross-validation</li></ul>
</li>

<li style="text-align: justify"><a href="https://github.com/KevinMMendez/cimcb_lite"><code>cimcb_lite</code></a>: a library of helpful functions provided by the authors</li>
</ul>

<p style="text-align: justify"><strong>Run the cell by clicking anywhere in the cell (the cell will be surrounded by a blue box) and then clicking <kbd>Run</kbd> in the Menu.</strong> <br>
When successfully executed the cell will print <code>All packages successfully loaded</code> in the notebook below the cell.</p>
</div>

In [ ]:
import numpy as np
import pandas as pd

from beakerx.object import beakerx
from sklearn.model_selection import train_test_split

import cimcb_lite as cb

beakerx.pandas_display_table()  # by default display pandas tables as BeakerX interactive tables

print('All packages successfully loaded')

<div style="background-color:rgb(255, 250, 250); padding:5px; padding-left: 1em; padding-right: 1em;">
    
<h2 id="2loaddataandpeaksheet" style="text-align: justify">2. Load Data and Peak sheet</h2>

<p style="text-align: justify">This workflow requires data to be uploaded as a Microsoft Excel file, using the <a href="https://en.wikipedia.org/wiki/Tidy_data">Tidy Data</a> framework (i.e. each column is a variable, and row is an observation). As such, the Excel file should contain a <em>Data Sheet</em> and <em>Peak Sheet</em>. The <em>Data Sheet</em> contains all the metabolite concentrations and metadata associated with each observation (requiring the inclusion of the columns: <em>Idx</em>, <em>SampleID</em>, and <em>Class</em>). The <em>Peak Sheet</em> contains all the metadata pertaining to each measured metabolite (requiring the inclusion of the columns: <em>Idx</em>, <em>Name</em>, and <em>Label</em>). Please inspect the <a href="GastricCancer_NMR.xlsx">Excel file</a> used in this tutorial before proceeding. </p>

<p style="text-align: justify">The code cell below loads the <em>Data</em> and <em>Peak</em> sheets from an Excel file, using the CIMCB helper function <code>load_dataXL()</code>. When this is complete, you should see confirmation that Peak (stored in the <code>Peak</code> worksheet in the Excel file) and Data (stored in the <code>Data</code> worksheet in the Excel file) tables have been loaded:</p>

<pre style="text-align: justify"><code class="text language-text">Loadings PeakFile: Peak
Loadings DataFile: Data
Data Table &amp; Peak Table is suitable.
TOTAL SAMPLES: 140 TOTAL PEAKS: 149
Done!
</code></pre>

<p style="text-align: justify">Once loaded, the data is available for use in <a href="https://swcarpentry.github.io/python-novice-gapminder/02-variables/"><em>variables</em></a> called <code>dataTable</code> and <code>peakTable</code>.</p>

</div>

In [ ]:
# The path to the input file (Excel spreadsheet)
filename = 'GastricCancer_NMR.xlsx'

# Load Peak and Data tables into two variables
dataTable, peakTable = cb.utils.load_dataXL(filename, DataSheet='Data', PeakSheet='Peak') 

<div style="background-color:rgb(255, 250, 250); padding:5px; padding-left: 1em; padding-right: 1em;">

<h3 style="text-align: justify" style="text-align: justify"> 2.1 Display the <code>Data</code> table </h3>

<p style="text-align: justify">
    The <code>dataTable</code> table can be displayed interactively so we can inspect and check the imported values. To do this, we use the <code>display()</code> function. For this example the imported data consists of 140 samples and 149 metabolites. 
</p>

<p style="text-align: justify">Note that each row describes a single urine sample, where:</p>

<ul>
    <li style="text-align: justify">Columns <b>M1</b> ... <b>M149</b> descibe metabolite concentrations.</li>
    <li style="text-align: justify">Column <b>SampleType</b> indicates whether the sample was a pooled QC or a study sample.</li>
    <li style="text-align: justify">Column <b>Class</b> indicates the clincal outcome observed for that individual: <i>GC</i> = Gastric Cancer , <i>BN</i> = Benign Tumor , <i>HE</i> = Healthy Control</li>.
</ul>

</div>

In [ ]:
display(dataTable) # View and check the dataTable 

<div style="background-color:rgb(255, 250, 250); padding:5px; padding-left: 1em; padding-right: 1em;">
<h3 id="22displaythepeaktable"  style="text-align: justify">2.2 Display the <code>Peak</code> table</h3>

<p style="text-align: justify">The <code>peakTable</code> table can also be displayed interactively so we can inspect and check the imported values. To do this, we again use the <code>display()</code> function. For this example the imported data consists of 149 metabolites (the same as in the <code>dataTable</code> data)</p>

<p style="text-align: justify">Each row describes a single metabolite, where</p>

<ul>
<li style="text-align: justify">Column <strong>Idx</strong> is a unique metabolite index.</li>

<li style="text-align: justify">Column <strong>Name</strong> is the column header corresponding to this metabolite in the <code>dataTable</code> table.</li>

<li style="text-align: justify">Column <strong>Label</strong> provides a unique name for the metabolite (or a <code>uNNN</code> identifier)</li>

<li style="text-align: justify">Column <strong>Perc_missing</strong> indicates what percentage of samples do not contain a measurement for this metabolite (missing data). </li>

<li style="text-align: justify">Column <strong>QC_RSD</strong> is a quality score representing the variation in measurements of this metabolite across all samples. </li>


</ul>
</div>

In [ ]:
display(peakTable) # View and check PeakTable

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">

<h2 id="3datacleaning" style="text-align: justify">3. Data Cleaning</h2>

<p style="text-align: justify">It is good practice to assess the quality of your data, and remove (clean out) any poorly measured metabolites, before performing any statistical or machine learning modelling <a href="https://link.springer.com/article/10.1007/s11306-018-1367-3">Broadhurst <em>et al.</em> 2018</a>.</a> For the Gastric Cancer NMR data set used in this example we have already calculated some basic statistics for each metabolite and stored them in the Peak table. In this notebook we keep only metabolites that meet the following criteria:</p>

<ul>
<li style="text-align: justify">a QC-RSD less than 20% </li>

<li style="text-align: justify">fewer than 10% of values are missing</li>
</ul>

<p style="text-align: justify">When the data is cleaned, the number of remaining peaks will be reported.</p>

</div>

In [ ]:
# Create a clean peak table 

rsd = peakTable['QC_RSD']  
percMiss = peakTable['Perc_missing']  
peakTableClean = peakTable[(rsd < 20) & (percMiss < 10)]   

print("Number of peaks remaining: {}".format(len(peakTableClean)))

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">

<h2 id="4pcaqualityassesment" style="text-align: justify">4. PCA - Quality Assessment</h2>

<p style="text-align: justify">To provide a multivariate assesment of the quality of the cleaned data set it is good practice to perform a simple  <a href="https://en.wikipedia.org/wiki/Principal_component_analysis">Principal Component Analysis</a> (PCA), after suitable <a href="https://doi.org/10.1186/1471-2164-7-142">transforming &amp; scaling</a>. The PCA score plot is typically labelled by sample type (i.e. quality control (QC) or biological sample (Sample)). Data of high quality will have QCs that cluster tightly compared to the biological samples <a href="https://link.springer.com/article/10.1007/s11306-018-1367-3">Broadhurst <em>et al.</em> 2018</a>. </p>

<p style="text-align: justify">First the metabolite data matrix is extracted from the <code>dataTable</code>, and transformed &amp; scaled:</p>

<ul>
<li style="text-align: justify">A new variable <code>peaklist</code> is created, to hold the names (M1...Mn) of the metabolites to be used in subsequent statistical analysis</li>

<li style="text-align: justify">The peak data for all samples, corresponding to this list, is extracted from the <code>dataTable</code> table, and placed in a matrix <code>X</code></li>

<li style="text-align: justify">The values in <code>X</code> are log-transformed (<code>Xlog</code>)</li>

<li style="text-align: justify">The helper function <code>cb.utils.scale()</code> is used to scale the log-transformed data (<code>Xscale</code>)</li>

<li style="text-align: justify">Missing values are imputed using a <em>k</em>-nearest neighbour approach (with three neighbours) to give the table <code>Xknn</code></li>
</ul>

<p style="text-align: justify">The transformed &amp; scaled dataset <code>Xknn</code> is used as input to PCA, using the helper function <code>cb.plot.pca()</code>. This returns plots of PCA scores and PCA loadings, for interpretation and quality assessment.</p>

</div>

In [ ]:
# Extract and scale the metabolite data from the dataTable 

peaklist = peakTableClean['Name']                   # Set peaklist to the metabolite names in the peakTableClean
X = dataTable[peaklist].values                      # Extract X matrix from dataTable using peaklist
Xlog = np.log10(X)                                  # Log scale (base-10)
Xscale = cb.utils.scale(Xlog, method='auto')        # methods include auto, pareto, vast, and level
Xknn = cb.utils.knnimpute(Xscale, k=3)              # missing value imputation (knn - 3 nearest neighbors)

print("Xknn: {} rows & {} columns".format(*Xknn.shape))

cb.plot.pca(Xknn,
            pcx=1,                                                  # pc for x-axis
            pcy=2,                                                  # pc for y-axis
            group_label=dataTable['SampleType'])                    # labels for Hover in PCA loadings plot

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">
    
<h2 id="5univariatestatisticsforcomparisonofgastriccancergcvshealthycontrolshe" style="text-align: justify">5. Univariate Statistics for comparison of Gastric Cancer (<code>GC</code>) vs Healthy Controls (<code>HE</code>)</h2>

<p style="text-align: justify">The data set uploaded into <code>dataTable</code> describes the <sup>1</sup>H-NMR urine metabolite profiles of individuals classified into three distinct groups: <code>GC</code> (gastric cancer), <code>BN</code> (benign), and <code>HE</code> (healthy). For this workflow we are interested in comparing only the differences in profiles between individuals classified as <code>GC</code> or <code>HE</code>.</p>

<p style="text-align: justify">The helper function <code>cb.utils.univariate_2class()</code> will take as input a data table where the observations represent data from two groups, and a corresponding table of metabolite peak information, and produce as output summary statistics of univariate comparisons between the two groups. The output is returned as a <code>pandas</code> dataframe, describing output from statistical tests such as Student's <em>t</em>-test and Shapiro-Wilks, and summaries of data quality, like the number and percentage of missing values.</p>

<p style="text-align: justify">First, we reduce the data in <code>dataTable</code> to only those observations for <code>GC</code> and <code>HE</code> samples, and we define the <code>GC</code> class to be a positive outcome, in the variable <code>pos_outcome</code>. Next, we pass the reduced dataset and the cleaned <code>peakTable</code> to <code>cb.utils.univariate_2class()</code>, and store the returned dataframe in a new variable called <code>statsTable</code>. This is then displayed as before for interactive inspection and interpretation.</p>

</div>

In [ ]:
# Select subset of Data for statistical comparison
dataTable2 = dataTable[(dataTable.Class == "GC") | (dataTable.Class == "HE")]  # Reduce data table only to GC and HE class members
pos_outcome = "GC" 

# Calculate basic statistics and create a statistics table.
statsTable = cb.utils.univariate_2class(dataTable2,
                                        peakTableClean,
                                        group='Class',                # Column used to determine the groups
                                        posclass=pos_outcome,         # Value of posclass in the group column
                                        parametric=True)              # Set parametric = True or False

# View and check StatsTable
display(statsTable)

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">

<p style="text-align: justify">It is useful to have this interactive view, but the output will disappear when we close the notebook. To store the output in a more persistent format, such as an Excel spreadsheet, we can use the <em>methods</em> that are built in to <code>pandas</code> dataframes.</p>

<p style="text-align: justify">To save a <code>pandas</code> dataframe to an Excel spreadsheet file as a single sheet, we use the dataframe's <code>.to_excel()</code><a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_excel.html">method</a>, and provide the name of the file we want to write to (and optionally a name for the sheet). We do not want to keep the dataframe's own index column, so we also set <code>index=False</code>.</p>

<p style="text-align: justify">The code in the cell below will write the contents of <code>statsTable</code> to the file <code>stats.xlsx</code>.</p>

</div>

In [ ]:
# Save StatsTable to Excel
statsTable.to_excel("stats.xlsx", sheet_name='StatsTable', index=False)
print("done!")

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">
    
<h2 style="text-align: justify"> 6. Machine Learning </h2>

<p style="text-align: justify">
The remainder of this tutorial will describe the use of a 2-class <a href="https://en.wikipedia.org/wiki/Partial_least_squares_regression">Partial Least Squares</a>-<a href="https://doi.org/10.1002/cem.713">Discriminant Analysis</a> (PLS-DA) model to identify metabolites which, when combined in a <a href="https://en.wikipedia.org/wiki/Linear_equation">linear equation</a>, are able to classify unknown samples as either <code>GC</code> and <code>HE</code> with a measurable degree of certainty.
</p>

<h3 style="text-align: justify"> 6.1 Splitting data into Training and Test sets. </h3>

<p style="text-align: justify">
Multivariate predictive models are prone to <a href="https://en.wikipedia.org/wiki/Overfitting">overfitting</a>. In order to provide some level of independent evaluation it is common practice to split the source data set into two parts: <strong>training set</strong> and <strong>test set</strong>. The model is then optimised using the training data and independently evaluated using the test data. The true effectiveness of a model can only be assessed using the test data (<a href="https://link.springer.com/article/10.1007/s11306-007-0099-6">Westerhuis <em>et al.</em> 2008</a>, <a href="https://doi.org/10.1007/s11306-012-0482-9">Xia <em>et al.</em> 2012</a>). It is vitally important that both the training and test data are equally representative of the the sample population (in our example the urine metabotype of <em>Gastric Cancer</em> and the urine metabotype of <em>Healthy Control</em>). It is typical to split the data using a ratio of 2:1 (&#x2154; training, &#x2153; test) using <a href="https://en.wikipedia.org/wiki/Stratified_sampling">stratified random selection</a>. If the purpose of model-building is exploratory, or sample numbers are small, this step is often ignored; however, care must be taken in interpreting a model that has not been tested on a dataset that is independent of the data it was trained on.
</p>  

<p style="text-align: justify">
We use the <code>dataTable2</code> dataframe created above, which contains a subset of the complete data suitable for a 2-class comparision (<code>GC</code> vs <code>HE</code>). Our goal is to split this dataframe into a <em>training</em> subset (<code>dataTrain</code>) which will be used to train our model, and a <em>test</em> set (<code>dataTest</code>), which will be used to evaluate the trained model. We will split the data such that number of <em>test</em> set samples is 25% of the the total. To do this, we will use the <a href="https://scikit-learn.org/stable/"><code>scikit-learn</code> module</a>'s <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html"><code>train_test_split()</code> function</a>.
</p>

<p style="text-align: justify">
First, we need to ensure that the sample split - though random - is <em>stratified</em> so that the class membership is <em>balanced</em> to the same proportions in both the test and training sets. In order to do this, we need to supply a binary vector indicating stratification group membership.
</p>

<p style="text-align: justify">
The <code>train_test_split()</code> function expects a <em>binary</em> (<code>1</code>/<code>0</code>) list of <em>positive</em>/<em>negative</em> <strong>outcome</strong> indicators, not the <code>GC</code>/<code>HE</code> classes that we have. We convert the class information for each sample in <code>dataTable2</code> into <code>Y</code>, a list of <code>1</code>/<code>0</code> values, in the code cell below.
</p>

</div>

In [ ]:
# Create a Binary Y vector for stratifiying the samples
outcomes = dataTable2['Class']                                  # Column that corresponds to Y class (should be 2 groups)
Y = [1 if outcome == 'GC' else 0 for outcome in outcomes]       # Change Y into binary (GC = 1, HE = 0)  
Y = np.array(Y)                                                 # convert boolean list into to a numpy array

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">

<p style="text-align: justify">Now that we have the dataset (<code>dataTable2</code>) and the list of binary outcomes (<code>Y</code>) for stratification, we can use the <code>train_test_split()</code> function in the code cell below.</p>

<p style="text-align: justify">Once the training and test sets have been created, summary output will be printed:</p>

<pre style="text-align: justify"><code class="text language-text">DataTrain = 62 samples with 32 positive cases.
DataTest = 21 samples with 11 positive cases.
</code></pre>

<p style="text-align: justify">Two new dataframes and two new lists are created:</p>

<ul>
<li style="text-align: justify"><code>dataTrain</code>: training data set (dataframe)</li>

<li style="text-align: justify"><code>dataTest</code>: test dataset (dataframe)</li>

<li style="text-align: justify"><code>Ytrain</code>: known outcomes for the training dataset</li>

<li style="text-align: justify"><code>Ytest</code>: known outcomes for the test dataset</li>
</ul>

</div>

In [ ]:
# Split dataTable2 and Y into train and test (with stratification)
dataTrain, dataTest, Ytrain, Ytest = train_test_split(dataTable2, Y, test_size=0.25, stratify=Y,random_state=10)

print("DataTrain = {} samples with {} positive cases.".format(len(Ytrain),sum(Ytrain)))
print("DataTest = {} samples with {} positive cases.".format(len(Ytest),sum(Ytest)))

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">
  
<h3 style="text-align: justify"> 6.2. Determine optimal number of components for PLS-DA model </h3>

<p style="text-align: justify">The most common method to determine the optimal PLS-DA model configuration without overfitting is to use <a href="https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html">k-fold cross-validation</a>. For PLS-DA, this will be a linear search of models having <i>$1$ to $N$</i> latent variables (components).
    
First, each PLS-DA configuration is trained using all the available data (<code>XTknn</code> and <code>Ytrain</code>). The generalised predictive ability of that model is then  evaluated using the same data - typically by calculating the <a href="https://en.wikipedia.org/wiki/Coefficient_of_determination">coefficient of determination</a> $R^2$. This will generate $N$ evaluation scores ($R^2_1,R^2_2 ... R^2_N$).

The training data is then split into <i>k</i> equally sized subsets (folds). For each of the PLS-DA configurations, $k$ models are built, such that each model is trained using $k-1$ <i>folds</i> and the remaining 1-fold is applied to the model and model predictions are recorded. The modeling process is implemented such than that after $k$ models each fold will have been *'held-out'* only once.

The generalised predictive ability of the model is then evaluated by comparing the *'held-out'* model predictions to the expected classification (cross-validated coefficient of determination - $Q^2$). This will generate $N$ <i>cross-validated</i> evaluation scores scores ($Q^2_1,Q^2_2 ... Q^2_N$). If the values for $R^2$ and $Q^2$ are plotted against model complexity (number of latent variables), typically the value of $Q^2$ will be seen to rise and then fall. The point at which the $Q^2$ value begins to diverge from the $R^2$ value is considered the point at which the optimal number of components has been met without overfitting.</p>

<p style="text-align: justify">In this section, we perform 5-fold cross-validation using the training set we created above (<code>dataTrain</code>) to determine the optimal number of components to use in our PLS-DA model.</p>

<p style="text-align: justify">First, in the cell below we extract and scale the training data in <code>dataTrain</code> the same way as we did for PCA quality assessment in section 4 (log-transformation, scaling, and k-nearest-neighbour imputation of missing values).</p>

</div>

In [ ]:
# Extract and scale the metabolite data from the dataTable
peaklist = peakTableClean['Name']                           # Set peaklist to the metabolite names in the peakTableClean
XT = dataTrain[peaklist]                                    # Extract X matrix from DataTrain using peaklist
XTlog = np.log(XT)                                          # Log scale (base-10)
XTscale = cb.utils.scale(XTlog, method='auto')              # methods include auto, pareto, vast, and level
XTknn = cb.utils.knnimpute(XTscale, k=3)                    # missing value imputation (knn - 3 nearest neighbors)

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">
    
<p style="text-align: justify">Now we use the <code>cb.cross_val.kfold()</code> helper function to carry out 5-fold cross-validation of a set of PLS-DA models configured with different numbers of latent variables (1 to 6). This helper function is generally applicable, and the values being passed here are:</p>

<ul>
<li style="text-align: justify"><code>model</code>: the Python class describing the statistical model to train and validate. Here, this is <code>cb.model.PLS_SIMPLS</code>, a PLS model using the SIMPLS algorithm.</li>

<li style="text-align: justify"><code>X</code>: the training data set (<code>XTknn</code>)</li>

<li style="text-align: justify"><code>Y</code>: the known outcomes corresponding to the dataset in <code>X</code> (<code>Ytrain</code>)</li>

<li style="text-align: justify"><code>param_dict</code>: a dictionary describing key:value pairs where the key is a parameter that is passed to the model, and the value is a collection of individual values to be passed to that parameter.</li>

<li style="text-align: justify"><code>folds</code>: the number of folds in the cross-validation</li>

<li style="text-align: justify"><code>bootnum</code>: the number of bootstrap samples used in computing confidence intervals</li>
</ul>

<p style="text-align: justify">The <code>cb.cross_val.kfold()</code> function returns an object that we store in the <code>cv</code> variable. To actually run the cross-validation, we call the <code>cv.run()</code> method of this object. When the cell is run, a progress bar will appear:</p>

<pre style="text-align: justify"><code>Kfold: 100%|██████████| 100/100 [00:02&lt;00:00, 33.71it/s]
</code></pre>

</div>

In [ ]:
# initalise cross_val kfold (stratified) 
cv = cb.cross_val.kfold(model=cb.model.PLS_SIMPLS,                   # model; we are using the PLS_SIMPLS model
                        X=XTknn,                                 
                        Y=Ytrain,                               
                        param_dict={'n_components': [1,2,3,4,5,6]},  # The numbers of latent variables to search                
                        folds=5,                                     # folds; for the number of splits (k-fold)
                        bootnum=100)                                 # num bootstraps for the Confidence Intervals

# run the cross validation
cv.run()  

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">

<p style="text-align: justify">The object stored in the <code>cv</code> variable also has a <code>.plot()</code> method, which renders two views of $R^2$ and $Q^2$ statistics: difference ($R^2 - Q^2$), and absolute values of both metrics against the number of components, to aid in selecting the optimal number of components.</p>

<p style="text-align: justify">The point at which the $Q^2$ value begins to diverge from the $R^2$ value is considered to be the point at which the optimal number of components has been met without overfitting. In this case, the plots clearly indicate that the optimal number of latent variables in our model is two.</p>

</div>

In [ ]:
cv.plot() # plot cross validation statistics


<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">
    
<h3 style="text-align: justify"> 6.3 Train and evaluate PLS-DA model </h3>

<p style="text-align: justify">Now we have determined that the optimal number of components for this example data set is 2, we create a PLS-DA model with 2 latent variables, and evaluate its predictive ability. The <a href="https://doi.org/10.1016/0169-7439(93)85002-X">implementation of PLS</a> we use is the <code>PLS_SIMPLS</code> class from the CIMCB helper module. We first create a PLS model object with two components, in the variable <code>modelPLS</code>:</p>

</div>

In [ ]:
modelPLS = cb.model.PLS_SIMPLS(n_components=2)  # Initalise the model with n_components = 2

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">

<p style="text-align: justify">Next we fit the model on the <code>XTknn</code> training dataset, with the values in <code>Ytrain</code> as the known response variables. We do this by calling the model's <code>.train()</code> method, with the predictor and response variables.</p>

<p style="text-align: justify">This returns a list of values that are the <em>predicted</em> response variables, after model fitting.</p>

</div>

In [ ]:
Ypred = modelPLS.train(XTknn, Ytrain)  # Train the model 

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">

<p style="text-align: justify">Finally, we call the trained model's <code>.evaluate()</code> method, passing a <em>classification cutoff score</em> from which a standard set of model evaluations will be calculated from the model predictions (<a href="https://en.wikipedia.org/wiki/Coefficient_of_determination">$R^2$</a>, <a href="https://en.wikipedia.org/wiki/Mann%E2%80%93Whitney_U_test">Mann-Whitney p-value</a>, <a href="https://doi.org/10.1007/s11306-012-0482-9">Area under ROC curve</a>, <a href="https://en.wikipedia.org/wiki/Accuracy_and_precision">Accuracy, Precision</a>, <a href="https://en.wikipedia.org/wiki/Sensitivity_and_specificity">Sensitivity, Specificity</a>). The model perfomance is also visualised using the following plots:</p>

<ul>
<li style="text-align: justify">a <a href="https://www.data-to-viz.com/graph/violin.html">violin plot</a> showing the distributions of negative and positive responses as violin plots and box-whisker plots, with an overlay of the predicted cutoff score that discriminates between classes (dashed line).</li>

<li style="text-align: justify">a <a href="https://books.google.com.au/books?id=7WBMrZ9umRYC">probability density function</a> plot for each response type, with overlaid predicted cutoff score (dashed line)</li>

<li style="text-align: justify"><a href="https://doi.org/10.1007/s11306-012-0482-9">ROC curve</a> for the classifier, with 95% confidence interval (lighter shaded area), and the performance indicated with 95% CI.</li>
</ul>

<p style="text-align: justify">From these plots and the table we find that the trained classifier performs acceptably well.</p>

</div>

In [ ]:
# Evaluate the model 
modelPLS.evaluate(cutoffscore=0.5)  

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">
    
<h3 style="text-align: justify"> 6.4. Perform a permutation test for the PLS-DA model </h3>

<p style="text-align: justify">The reliability of our trained model can be assessed using a <a href="https://en.wikipedia.org/wiki/Resampling_(statistics)"><em>permutation test</em></a>. In this test, the original data is randomised (<em>permuted</em> or 'shuffled') so that the predictor variables and response variables are mixed, and a new model is then trained and tested on the shuffled data. This is repeated many times so that the behaviour of models constructed from "random" data can be fairly assessed.</p>

<p style="text-align: justify">We can be confident that our model is being trained on relevant and meaningful features of the original dataset if the $R^2$ and $Q^2$ values generated from these models (with randomised data) are much lower than those found for our model trained on the original data.</p>

<p style="text-align: justify">The PLS model we are using from the CIMCB module has a <code>.permutation_test()</code> method that can perform this analysis for us. It returns a pair of graphs that can be used to interpret model performance.</p>

<ul>
<li style="text-align: justify">$R^2$ and $Q^2$ against "correlation of permuted data against original data"</li>

<li style="text-align: justify">probability density functions for $R^2$ and $Q^2$, with the $R^2$ and $Q^2$ values found for the model trained on original data presented as ball-and-stick.</li>
</ul>

<p style="text-align: justify">We see that the models trained on randomised/shuffled data have much lower $R^2$ and $Q^2$ values than the models trained on the original data, so we can be confident that the model represents meaningful features in the original dataset.</p>

</div>

In [ ]:
modelPLS.permutation_test(nperm=100) #nperm refers to the number of permutations

<a id='6.5'></a>
<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">
    
<h3 style="text-align: justify"> 6.5. Plot latent variable projections for PLS-DA model </h3>

<p style="text-align: justify">The PLS model also provides a <code>.plot_projections()</code> method, so we can visually inspect characteristics of the fitted latent variables. This returns a grid of plots:</p>

<ul>
<li style="text-align: justify">The diagonal shows probability density functions of each latent variable (LV) for each response class. The first latent variable (LV1) is at the top left of the plot.</li>

<li style="text-align: justify">The upper triangle shows ROC curves for each optimal discriminating pairwise combination of LVx and LVy scores</li>

<li style="text-align: justify">The lower triangle shows scatterplots of the scores for LVy against LVx, with a solid line indicating the direction of maximum discrimination</li>
</ul>

<p style="text-align: justify">Where only one latent variable is fitted, a similar plot is produced to that with the <code>.evaluate()</code> method, with the addition of a scatterplot of latent variable scores)</p>

</div>

In [ ]:
modelPLS.plot_projections(label=dataTrain[['Idx','SampleID']], size=12) # size changes circle size

<a id='6.6'></a>
<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">

<h3 style="text-align: justify"> 6.6. Plot feature importance (Coefficient plot and VIP) for PLS-DA model </h3>

<p style="text-align: justify">Now that we have built a model and established that it represents meaningful features of the dataset, we determine the importance of specific peaks to the model's discriminatory power.</p>

<p style="text-align: justify">To do this, in the cell below we use the PLS model's <code>plot_featureimportance()</code> method to render scatterplots of the <a href="https://doi.org/10.6084/m9.figshare.5696494.v3">PLS regression <em>coefficient</em> values</a> for each metabolite, and <a href="https://books.google.com.au/books?id=58qLBQAAQBAJ"><em>Variable Importance in Projection</em></a> (VIP) plots. The coefficient values provide information about the contribution of the peak to either a negative or positive classification for the sample, and peaks with VIP greater than unity (1) are considered to be "important" in the model.</p>

<p style="text-align: justify">We could generate these plots for the model as it was trained, but we would prefer to have an estimate of the robustness of these values, so we generate <a href="https://cds.cern.ch/record/526679/files/0412042312_TOC.pdf">bootstrapped confidence intervals</a> with the model's <code>.calc_bootci()</code> method. Any metabolite coefficient with a confidence interval crossing the zero line is considered non-significant, and thus not "important" to the model.</p>

<p style="text-align: justify">The <code>.plot_featureimportance()</code> method renders the two scatterplots, and also returns a new dataframe reporting these values, and their confidence intervals, which we capture in the variable <code>peakSheet</code>. </p>

</div>

In [ ]:
# Calculate the bootstrapped confidence intervals 
modelPLS.calc_bootci(type='bca', bootnum=200)                # decrease bootnum if it this takes too long on your machine

# Plot the feature importance plots, and return a new Peaksheet 
peakSheet = modelPLS.plot_featureimportance(peakTableClean,
                                            peaklist,
                                            ylabel='Label',  # change ylabel to 'Name' 
                                            sort=False)      # change sort to False

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">
    
<h3 style="text-align: justify"> 6.7. Test model with new data (using test set from section 6.1) </h3>

<p style="text-align: justify">So far, we have trained and tested our PLS classifier on a single training dataset. This risks <em>overfitting</em> as we could be optimising the performance of the model on this dataset such that it cannot <em>generalise</em>, in the sense that it may not perform as well on a dataset that it has not already seen.</p>

<p style="text-align: justify">To see if the model can <em>generalise</em>, we must test our trained model using a new dataset that it has not already encountered. In section 6.1 we divided our original complete dataset into four components: <code>datatrain</code>, <code>Ytrain</code>, <code>dataTest</code> and <code>Ytest</code>. Our trained model has not seen the <code>dataTest</code> and <code>Ytest</code> values that we have <em>held out</em>, so these can be used to evaluate model preformance on new data.</p>

<p style="text-align: justify">We begin by transforming and scaling this <em>holdout</em> dataset in the same way as we did for the training data. To do this, we first find the mean and variance of our transformed training data set <code>XTlog</code> with the <code>cb.utils.scale()</code> function, so that we can use these values to scale the <em>holdout</em> data.</p>

</div>

In [ ]:
# Get mu and sigma from the training dataset to use for the Xtest scaling
mu, sigma  = cb.utils.scale(XTlog, return_mu_sigma=True) 

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">
    
<p style="text-align: justify">Next, we extract the peak data for our holdout <code>dataTest</code> set, and put this in the variable <code>XV</code>. As before, we take the log transform (<code>XVlog</code>), scale the data in the same way as the training data (<code>XVscale</code>; note that we specify <code>mu</code> and <code>sigma</code> as calculated above), and impute missing values to give the final <em>holdout</em> test set <code>XVknn</code>.</p>

</div>

In [ ]:
# Pull of Xtest from DataTest using peaklist ('Name' column in PeakTable)
peaklist = peakTableClean.Name 
XV = dataTest[peaklist].values

# Log transform, unit-scale and knn-impute missing values for Xtest
XVlog = np.log(XV)
XVscale  = cb.utils.scale(XVlog, method='auto', mu=mu, sigma=sigma) 
XVknn = cb.utils.knnimpute(XVscale, k=3)

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">

<p style="text-align: justify">Now we predict a new set of response variables from <code>XVknn</code> as input, using our trained model and its <code>.test()</code> method, and then evaluate the performance of model prediction against the known values in <code>Ytest</code> using the <code>.evaluate()</code> method (as in section 6.3).</p>

<p style="text-align: justify">Three plots are generated, showing comparisons of the performance of the model on training and holdout test datasets.</p>

<ul>
<li style="text-align: justify">a violin plot showing the distribution of known positive and negative in both training and test sets, and the class cut-off (dotted line)</li>

<li style="text-align: justify">probability density functions for positive and negative classes in the training and test sets (the training set datapoints are rendered as more opaque than the test set data in this figure)</li>

<li style="text-align: justify">ROC curves of model performance on training and test sets</li>
</ul>

<p style="text-align: justify">A table of performance metrics for both datasets is shown below the figures.</p>

</div>

In [ ]:
# Calculate Ypredicted score using modelPLS.test
YVpred = modelPLS.test(XVknn)

# Evaluate Ypred against Ytest
evals = [Ytest, YVpred]    # alternative formats: (Ytest, Ypred) or np.array([Ytest, Ypred])
#modelPLS.evaluate(evals, specificity=0.9)
modelPLS.evaluate(evals, cutoffscore=0.5) 

<a id='6.8'></a>
<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">
    
<h3 style="text-align: justify"> 6.8. Export results to Excel </h3>

<p style="text-align: justify">Finally, we will save our results in a persistent Excel spreadsheet.</p>

<p style="text-align: justify">Unlike section 5, we want to save two sheets in a single Excel workbook called <code>modelPLS.xlsx</code>. We want to save one sheet showing the holdout test data (with results from <code>YVpred</code>), and a separate sheet showing the peaks with their residual coefficients and VIP scores. </p>

<p style="text-align: justify">Firstly, we generate a dataframe containing the test dataset and the model's predictions. This will have columns for</p>

<ul>
<li style="text-align: justify"><code>idx</code>: sample index</li>

<li style="text-align: justify"><code>SampleID</code>: sample ID</li>

<li style="text-align: justify"><code>class</code>: sample class (<code>GC</code> or <code>HE</code>)</li>

<li style="text-align: justify"><code>YPred</code>: predicted response variable from the trained model</li>
</ul>

</div>

In [ ]:
# Save DataSheet as 'Idx', 'SampleID', and 'Class' from DataTest
dataSheet = dataTest[["Idx", "SampleID", "Class"]].copy() 

# Add 'Ypred' to Datasheet
dataSheet['Ypred'] = YVpred 
 
display(dataSheet) # View and check the dataTable 

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">

<p style="text-align: justify">In section 5 we saved a single dataframe to an Excel workbook, as a single worksheet. Here, we want to save two worksheets. This means we can't use the <code>.to_excel()</code> method of a dataframe directly to write twice to the same file. Instead, we must create a <code>pd.ExcelWriter</code> object, and add each dataframe in turn to this object. When we are finished adding datframes, we can use the object's <code>.save()</code> method to write the Excel workbook with several worksheets (one per dataframe) to a single file.</p>

</div>


In [ ]:
# Create an empty excel workbook
writer = pd.ExcelWriter("modelPLS.xlsx")     # provide the filename for the Excel file

# Add each dataframe to the workbook in turn, as a separate worksheet
dataSheet.to_excel(writer, sheet_name='Datasheet', index=False)
peakSheet.to_excel(writer, sheet_name='Peaksheet', index=False)

# Write the Excel workbook to disk
writer.save()

print("Done!")

<div style="background-color:rgb(255, 250, 250); padding:10px; padding-left: 1em; padding-right: 1em;">

<p style="text-align: justify"> Congratulations! You have reached the end of tutorial 1. </p>

</div>